In [ ]:
import math

import pandas as pd
df1 = pd.read_csv('./data/es_algolia_788741_26april.tsv', sep='\t', header=0)
df2 = pd.read_csv('./data/im_logs_788741.tsv', sep='\t', header=0)

df1.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df2.applymap(lambda x: x.strip() if isinstance(x, str) else x)




s = df1["Query"].isna()

df1["Query"].ffill(inplace=True)


df1['algolia_sugg_rank'] = df1.groupby('Query')['Query'].rank(method='first')
df1['es_sugg_rank'] = df1.groupby('Query')['Query'].rank(method='first')






df2['sugg_count'] = df2.groupby('Query')['Query'].transform('count')
df2.sort_values(['Query'])
df2['order_count'].fillna(0, inplace = True)

df2['sugg_rank'] = df2.sort_values(['Query', 'order_count', 'SEARCHES'], ascending=False).groupby(['Query']).cumcount() + 1
df2.sort_values(['Query', 'order_count'], ascending = [1,0])

df2 = df2[df2['sugg_rank'] <= 10]
#print(df2.count())
#df2['suggestion']


df2_sel_col = df2[['Query', 'suggestion', 'sugg_rank']].rename(columns = {'suggestion' : 'ground_truth_product', 'sugg_rank': 'gt_product_rank'})

df1 = df1.merge(df2_sel_col,
               how='left',
               left_on=['Query', 'algolia_sugg_rank'],
               right_on=['Query', 'gt_product_rank']
               )





df1_out = df1.drop(['algolia_sugg_rank', 'es_sugg_rank', 'gt_product_rank'], axis=1, inplace=False).rename(columns = {'Algolia Results': 'algolia', 'Elastic-search results': 'es'})

df1_algolia = df1_out.groupby('Query').agg({'algolia':lambda x: list(x)})
df1_es = df1_out.groupby('Query').agg({'es':lambda x: list(x)})
df1_gt = df1_out.groupby('Query').agg({'ground_truth_product':lambda x: list(x)})

dict_algo = df1_algolia.to_dict()#['Algolia Results'].keys()#['paneer']
dict_es = df1_es.to_dict()#['Algolia Results'].keys()#['paneer']
dict_gt = df1_gt.to_dict()#['Algolia Results'].keys()#['paneer']

dict_all = dict_algo
dict_all.update(dict_es)
dict_all.update(dict_gt)
#dict_all.keys()

avg_recall_es = 0
avg_recall_algolia = 0
count = 0
skip_count = 0
new_recall_dict = {}
avg_ndcg_algolia = 0
avg_ndcg_es = 0

for key in dict_algo['algolia']:
    gt_list = [str(p).strip() for p in dict_all['ground_truth_product'][key] if str(p) != 'nan']
    if len(set(gt_list)) == 0:
        skip_count += 1
        continue

    algolia_list = [str(p).strip() for p in dict_all['algolia'][key] if str(p) != 'nan']
    es_list = [str(p).strip() for p in dict_all['es'][key] if str(p) != 'nan']
    #print(key)
    if key == 'coconut':
        # print(algolia_list)
        # print(gt_list)
        # print(es_list)
        # print(len(set(algolia_list)), len(set(es_list)), len(set(gt_list)))
    recall_algolia = len(set(gt_list).intersection(set(algolia_list)))*100/len(set(gt_list))
    avg_recall_algolia += recall_algolia

    recall_es =  len(set(gt_list).intersection(set(es_list)))*100/len(set(gt_list))
    avg_recall_es += recall_es
    #print(recall_algolia, recall_es)
    if not (recall_algolia == 0 and recall_es == 0):
        count += 1
    else:
        skip_count += 1

    #add info to output dictionary
    new_recall_dict[key] = {'recall_algolia_in_gt_top10': recall_algolia,
                            'recall_es_in_gt_top10': recall_es}

    # ndcg calculation
    relevance_list_algolia = []
    relevance_list_es = []
    for item in algolia_list:
        if item in gt_list:
            gt_index = gt_list.index(item)
        else:
            gt_index = 10
        if gt_index < 0:
            gt_index = 10
        relevance_list_algolia.append(10-gt_index)

    for item in es_list:
        if item in gt_list:
            gt_index = gt_list.index(item)
        else:
            gt_index = 10

        if gt_index < 0:
            gt_index = 10
        relevance_list_es.append(10-gt_index)

    algolia_dcg = 0
    es_dcg = 0
    algolia_idcg = 0
    es_idcg = 0
    for i in range(len(relevance_list_algolia)):
        algolia_dcg += relevance_list_algolia[i]/math.log2(i+2)
        algolia_idcg += (len(relevance_list_algolia) - i)/math.log2(i+2)

    for i in range(len(relevance_list_es)):
        es_dcg += relevance_list_es[i]/math.log2(i+2)
        es_idcg += (len(relevance_list_es) - i)/math.log2(i+2)


    avg_ndcg_algolia += algolia_dcg/algolia_idcg
    avg_ndcg_es += es_dcg/es_idcg

avg_recall_algolia = avg_recall_algolia/count
avg_recall_es = avg_recall_es/count
avg_ndcg_algolia = avg_ndcg_algolia/count
avg_ndcg_es = avg_ndcg_es/count

print('count, skip_count:', count, skip_count)
print('recall_algolia:', avg_recall_algolia)
print('recall_es:', avg_recall_es)
print('ndcg_algolia:', avg_ndcg_algolia)
print('ndcg_es:', avg_ndcg_es)

new_recall_df = pd.DataFrame.from_dict(new_recall_dict).T
new_recall_df['Query'] = new_recall_df.index
#new_recall_df

df1_out = df1_out.merge(new_recall_df,
               how='left',
               left_on=['Query'],
               right_on=['Query']
               )

#df1_out.to_csv('./data/im_poc_gt_788741_26apr_2days.tsv', sep = "\t")

